In [1]:
!pip install crewai


  Using cached crewai-0.86.0-py3-none-any.whl.metadata (19 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached auth0_python-4.7.2-py3-none-any.whl.metadata (8.9 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached crewai_tools-0.17.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached instructor-1.7.0-py3-none-any.whl.metadata (17 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached regex-

In [2]:
!pip install --q "crewai[tools]"

In [97]:
!pip install python-dotenv




In [98]:
!python -m venv venv


In [99]:
!venv\Scripts\activate


In [100]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, DallETool
from langchain_openai import ChatOpenAI

In [101]:
# Inicializar o modelo da OpenAI
import os
gpt_4o_mini = ChatOpenAI(model_name='gpt-4o-mini')

In [102]:
# Ferramenta de pesquisa na internet
search_tool = SerperDevTool()

# Ferramenta de criação de imagem
dalle_tool = DallETool(size='1792x1024')

#AGENTES

- Pesquisador da Web
- Escritor
- Roteirista
- Especialista em SEO
- Engenheiro de prompts para imagens
- Designer de imagem
- Revisor

In [103]:
# 1. Agente pesquisador de tema
pesquisador = Agent(
    role='Pesquisador do Tema',
    goal='pesquisar informações detalhadas sobre {tema} na internet',
    backstory=(
      "Você é especialista em pesquisa com habilidades aguçadas para"
      "encontrar informações detalhadas e valiosas na internet"
),
    verbose=True,
    memory=True,
    tools=[search_tool],
    # Aqui, vamos trabalhar com o Gemini.
    llm=gpt_4o_mini
)


In [104]:
escritor_titulos = Agent(
    role='Escritor de Títulos',
    goal='Escrever títulos de alta qualidade sobre o {tema} definido',
    backstory=(
      "Você é um escritor profissional com habilidades excepcionais"
      "na criação de títulos criativos, informativos e persuasivos"
      "para diferentes públicos e finalidades."
    ),
    verbose=True,
    memory=True,
    llm=gpt_4o_mini
)

In [105]:
roteirista = Agent(
    role='Roteirista Criativo',
    goal='Criar roteiros envolventes e criativos para vídeos sobre o {tema} definido',
    backstory=(
      "Você é um roteirista experiente com um talento especial"
      "para desenvolver narrativas envolventes e cativantes."
      "Você domina a arte de contar histórias que capturam a atenção."
    ),
    verbose=True,
    memory=True,
    llm=gpt_4o_mini
)

In [106]:
especialista_seo = Agent(
    role='Especialista em SEO para Youtube',
    goal='Otimizar conteúdo para mecanismos de busca, aumentando a visibilidade e tráfego orgânico',
    backstory=(
        "Você é um especialista em otimização para mecanismos de busca (SEO)"
        "com vasto conhecimento em técnicas de SEO on-page e off-page."
        "Você está sempre atualizado com as últimas tendências e algoritmos do Google."
    ),
    verbose=True,
    memory=True,
    tools=[search_tool],
    llm=gpt_4o_mini
)

In [107]:
engenheiro_prompts_imagens = Agent(
    role='Engenheiro de Prompts para o DALL-E',
    goal='Criar prompts detalhados e eficazes para gerar imagens de alta qualidade com IA, com base no {tema}',
    backstory=(
        "Você é um especialista em engenharia de prompts para geração de imagens com IA."
        "Possui conhecimento em diversas ferramentas e modelos de geração de imagens."
        "Sabe como criar prompts que resultam em imagens precisas e de alta qualidade."
    ),
    verbose=True,
    memory=True,
    tools=[search_tool],
    llm=gpt_4o_mini
)

In [108]:
designer_imagem = Agent(
    role='Gerador de Imaens com o DALL-E',
    goal='Criar imagens visualmente atraentes e impactantes sobre o {tema}, utilizando ferramentas de design e IA',
    backstory=(
        "Você é um designer de imagem com um talento especial para criar composições visuais"
        "que atraem e engajam o público. Você domina as ferramentas de design e geração de imagens com IA."
    ),
    verbose=True,
    memory=True,
    tools=[search_tool],
    llm=gpt_4o_mini
)

In [109]:
revisor = Agent(
    role='Revisor de Conteúdo',
    goal='Revisar e aperfeiçoar textos e outros materiais, garantindo a qualidade e precisão',
    backstory=(
        "Você é um revisor experiente e detalhista, com um olhar crítico e apurado para a qualidade"
        "e precisão de textos e outros materiais. Você garante que todo o conteúdo esteja impecável."
    ),
    verbose=True,
    memory=True,
    tools=[search_tool],
    llm=gpt_4o_mini
)

#TAREFAS

In [110]:
tarefa_pesquisa= Task(
    description=(
    "Pesquisar informações detalhadas sobre {tema} na internet."
    "Concentre-se em aspectos únicos e dados importantes que podem enriquecer o vídeo."
    "Todo o texto deverá estar em português do Brasil."
),
expected_output='Um documento com as principais informações e dados sobre o {tema} pesquisado.',
tools=[search_tool],
agent=pesquisador,
)

In [111]:
tarefa_titulos= Task(
    description=(
    "Criar títulos atraentes e otimozados sobre o tema: {tema}"
    "Certifique-se que o título seja cativante e otimizado para o SEO."
    "Todo o texto deverá estar em português do Brasil."
),
expected_output='Um título para o vídeo, otimizado para SEO, sobre o {tema}.',
agent=escritor_titulos,
)

In [112]:
tarefa_roteiro = Task(
    description=(
        "Criar um roteiro detalhado e envolvente para um vídeo sobre o tema: {tema}."
        "O roteiro deve incluir uma estrutura clara com introdução, desenvolvimento e conclusão."
        "Se necessário, inclua ideias criativas de imagem para manter o público engajado."
        "Use uma linguagem clara e direta, adequada para o público-alvo."
        "O roteiro deverá estar em português do Brasil."
    ),
    expected_output='Um roteiro completo e detalhado para um vídeo sobre o tema {tema}, com ideias criativas para engajamento.',
    agent=roteirista,
)

In [113]:
tarefa_seo = Task(
    description=(
        "Realizar uma análise de palavras-chave detalhada para otimizar o conteúdo sobre o tema: {tema}."
        "Identificar as principais palavras-chave relevantes para o tema, incluindo palavras-chave de cauda longa."
        "Analisar a concorrência para cada palavra-chave identificada."
        "Fornecer uma lista das palavras-chave mais relevantes e estratégicas para otimizar o conteúdo."
        "Todo o texto deverá estar em português do Brasil."
    ),
    expected_output='Uma lista de palavras-chave relevantes e estratégicas para otimizar o conteúdo sobre o tema {tema}, incluindo análise da concorrência.',
    agent=especialista_seo,
)

In [114]:
# Engenharia de prompt para gerar imagens

tarefa_prompts_imagens = Task(
    description=(
        "Criar um prompt detalhado e eficaz para gerar uma imagem relacionada ao tema: {tema}."
        "Conforme descrito no roteiro."
    ),
    expected_output='Um prompt de texto detalhado e eficaz para gerar uma imagem sobre o tema {tema}, com especificações claras sobre elementos visuais.',
    agent=engenheiro_prompts_imagens,
)

In [115]:
# Criação da imagem

tarefa_designer_imagem = Task(
    description=(
        "Gerar uma imagem usando o DALL-E usando o prompt fornecido pelo engenheiro de prompts."
        "Entregar a imagem em um formato comum de imagem (JPEG, PNG, etc.)."
    ),
    expected_output='Uma imagem gerada pronta para uso.',
    agent=designer_imagem,
)

In [116]:
# Tarefa de revisão e entrega

tarefa_revisao = Task(
    description=(
        "Revisar o texto, roteiro e/ou conteúdo, fornecido pelos outros agentes, sobre o tema: {tema}."
        "Verificar gramática, ortografia, pontuação, clareza, concisão, coerência e coesão do texto."
        "Incluir os links das imagens e preparar a versão final para entrega."
        "O resultado final deverá estar em português do Brasil."
    ),
    expected_output='Um texto revisado, corrigido e aprimorado sobre o tema {tema}, pronto para entrega ao usuário.',
    agent=revisor,
    output_file='output_teste.md' #configurando para salvar em um arquivo markdown.
)

#CREW

In [117]:
# Formatando a Crew

crew = Crew(
    agents=[
        pesquisador,
        escritor_titulos,
        roteirista,
        especialista_seo,
        engenheiro_prompts_imagens,
        designer_imagem,
        revisor
    ],
    tasks=[
        tarefa_pesquisa,
        tarefa_titulos,
        tarefa_roteiro,
        tarefa_seo,
        tarefa_prompts_imagens,
        tarefa_designer_imagem,
        tarefa_revisao
    ],

    process=Process.sequential # Processo sequencial   
)

2024-12-22 18:07:36,052 - 30812 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [118]:
tema_user = input("Qual o tema do vídeo que você deseja criar? ")

In [119]:
print(tema_user)

Inteligência Artificial no Futebol


In [120]:
# Executando o processo com o tema escolhido.

result = crew.kickoff(inputs={'tema':tema_user})

# Agent: Pesquisador do Tema
## Task: Pesquisar informações detalhadas sobre Inteligência Artificial no Futebol na internet.Concentre-se em aspectos únicos e dados importantes que podem enriquecer o vídeo.Todo o texto deverá estar em português do Brasil.


# Agent: Pesquisador do Tema
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Intelig\\u00eancia Artificial no Futebol\"}"
## Tool Output: 

Search results: Title: Como a IA está impactando treinos e jogos de futebol | by LabJor
Link: https://medium.com/labjorfaap/como-a-ia-est%C3%A1-impactando-treinos-e-jogos-de-futebol-401e01d896bd
Snippet: A Inteligência Artificial também chegou ao futebol. Mapeamento da atuação dos jogadores, análises de melhoria de desempenho e desenvolvimento de ...
---
Title: Clubes de futebol já começam a usar inteligência artificial para ...
Link: https://www.cnnbrasil.com.br/esportes/futebol/futebol-internacional/clubes-de-futebol-ja-comecam-a-usar-inteligencia-artificial-para-cacar-